# Import libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torch import nn
from torchvision import datasets, transforms

# Create "Dataset" class

In [ ]:
class Dataset():
    def __init__(self, csv_file, train, test_size, transform = None, labels=True):
        data = pd.read_csv(csv_file)
        key = int((1-test_size)*len(data))
        if train:
            self.data = pd.DataFrame(data.iloc[:key,:])
        else:
            self.data = pd.DataFrame(data.iloc[key:,:])
        self.transform = transform
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.labels:
            out = self.data.iloc[idx,1:].to_numpy().reshape((28,28)).astype("double")
            labels = self.get_label(idx)
        else:
            out = self.data.iloc[idx,:].to_numpy().reshape((28,28)).astype("double")
            labels = []
        out = out/255.
        if self.transform:
            out = self.transform(out)
        return out, labels

    def get_label(self,idx):
        return self.data.iloc[idx,0].item()

# Load data and transformations

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,),(0.5,))])

train_dataset = Dataset("Data/train.csv", train = True, test_size = 0.2, transform = transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, shuffle = True)
test_dataset = Dataset("Data/train.csv", train = False, test_size = 0.2, transform = transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = False)
train_iter = iter(train_loader)
test_iter = iter(test_loader)

# Prepare NN model

In [ ]:
class Classifier(nn.Module):
    def __init__(self, x_in, x1, x2, x_out):
        super().__init__()
        self.linear1 = nn.Linear(x_in, x1)
        self.linear2 = nn.Linear(x1,x2)
        self.linear3 = nn.Linear(x2,x_out)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x        

# Training settings

In [ ]:
model = Classifier(28*28, 135, 65, 10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

# Training

In [ ]:
epochs = 15
epoch_loss = []
epoch_acc = []
epoch_test_loss = []
epoch_test_acc = []
for e in range(epochs):
    actual_loss = 0.0
    actual_preds = 0.0
    test_loss = 0.0
    test_preds = 0.0
    for images, labels in train_loader:
        images = images.view(images.shape[0], -1).float()
        out = model(images)
        loss = criterion(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        actual_loss += loss.item()
        _, preds = torch.max(out,1)
        actual_preds += torch.sum(preds == labels)
    else:
        loss = actual_loss/len(train_loader)
        acc = actual_preds/len(train_loader)
        epoch_loss.append(loss)
        epoch_acc.append(acc)
        print('epoch {}'.format(e+1))
        print('Loss {}'.format(loss))
        print('Acc {}'.format(acc))
        
        for images, labels in test_loader:
            images = images.view(images.shape[0], -1).float()
            out = model(images)
            loss = criterion(out, labels)
            test_loss += loss.item()
            _, preds = torch.max(out,1)
            test_preds += torch.sum(preds == labels)
        else:
            test_loss = test_loss/len(test_loader)
            test_acc = test_preds/len(test_loader)
            epoch_test_loss.append(test_loss)
            epoch_test_acc.append(test_acc)
            print('Test_loss {}'.format(test_loss))
            print('Test_acc {}'.format(test_acc))

# Plot loss

In [ ]:
plt.plot(epoch_loss, color ='r', label = "Train Loss")
plt.plot(epoch_test_loss, color ='b', label = "Test loss")
plt.legend()
plt.show()

# Plot accuracy

In [ ]:
plt.plot(epoch_acc, color ='r', label = "Train accuracy")
plt.plot(epoch_test_acc, color ='b', label = "Test accuracy")
plt.legend()
plt.show()

# Function for converting image to normal version

In [ ]:
def im_convert(tensor):
  image = tensor.clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

# Plot sample predictions

In [ ]:
images, labels = train_iter.next()
images_flat = images.view(images.shape[0], -1).float()
out = model(images_flat)
_, preds = torch.max(out,1)
fig = plt.figure(figsize=(25, 4))

for idx in np.arange(20):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
    plt.imshow(im_convert(images[idx]))
    title = 'Predykcja: {}'.format(preds[idx]) 
    ax.set_title(title, color= ('g' if preds[idx]==labels[idx].item() else'r'))

# Load images for prediction

In [ ]:
new_dataset = Dataset("Data/test.csv", train = True, test_size = 0, transform = transform, labels = False)
new_loader = torch.utils.data.DataLoader(new_dataset, batch_size = 100, shuffle = False)
preds = []
for images,_ in new_loader:
    images = images.view(images.shape[0], -1).float()
    out = model(images)
    _, pred = torch.max(out, 1)
    preds = np.concatenate((preds,pred.numpy()))

# Save predictions to file

In [ ]:
preds = preds.astype(int)
y = np.zeros((28000, 2), dtype = int)
y[:,0] = range(1,28001)
y[:,1] = preds
df = pd.DataFrame(y,columns= ["ImageId","Label"])
df.to_csv("output_nn.csv", index = False)